In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
spark = SparkSession\
    .builder\
    .master('local')\
    .appName("lesson_13")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.json("/test/api_values.json")

In [4]:
df.head()

Row(date='2021-07-07', product_id=28521)

In [5]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- product_id: long (nullable = true)



In [6]:
df2 = spark.read.format('json').load("/test/api_values.json")

In [7]:
df2.first()

Row(date='2021-07-07', product_id=28521)

In [8]:
df.columns == df2.columns

True

In [9]:
df.subtract(df2).show()

+----+----------+
|date|product_id|
+----+----------+
+----+----------+



In [10]:
df == df2

False

In [11]:
df3 = spark.read\
    .option("multiline", "true")\
    .json("/test/multiline-zipcodes.json")

In [12]:
df3.show()

+-------------------+-------------+-----+-------+-----------+
|               City|Record number|State|ZipCode|ZipCodeType|
+-------------------+-------------+-----+-------+-----------+
|Paseo Costa Del Sur|            2|   PR|    707|   STANDARD|
|       BDA San Luis|           10|   PR|    709|   STANDARD|
+-------------------+-------------+-----+-------+-----------+



In [13]:
df1 = spark.read\
    .option("multiline", "true")\
    .json("/test/api_values1.json")
df2 = spark.read\
    .option("multiline", "true")\
    .json("/test/api_values2.json")

In [14]:
df4 = df1.union(df2)

In [15]:
df4.show()

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-07-05|     46298|
|2021-07-05|     48379|
|2021-07-06|     42662|
|2021-07-06|     45474|
+----------+----------+



In [16]:
df4.select('date','product_id').show()

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-07-05|     46298|
|2021-07-05|     48379|
|2021-07-06|     42662|
|2021-07-06|     45474|
+----------+----------+



In [17]:
del df4

In [18]:
df4 = spark.read\
    .option("multiline", "true")\
    .json(["/test/api_values1.json","/test/api_values2.json"])

In [19]:
df4.show()

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-07-05|     46298|
|2021-07-05|     48379|
|2021-07-06|     42662|
|2021-07-06|     45474|
+----------+----------+



In [21]:
schema = StructType([
    StructField("date", DateType(), False),
    StructField("product_id", IntegerType(), False)
    ])

In [22]:
df1 = spark.read\
    .option("multiline", "true")\
    .json("/test/api_values1.json")
df2 = spark.read\
    .option("multiline", "true")\
    .schema(schema)\
    .json("/test/api_values2.json")

In [23]:
df1.printSchema()
df2.printSchema()

root
 |-- date: string (nullable = true)
 |-- product_id: long (nullable = true)

root
 |-- date: date (nullable = true)
 |-- product_id: integer (nullable = true)



In [24]:
df1.show()
df2.show()

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-07-05|     46298|
|2021-07-05|     48379|
+----------+----------+

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-07-06|     42662|
|2021-07-06|     45474|
+----------+----------+



In [25]:
df2.createOrReplaceTempView('api_values_tables')

In [26]:
spark.sql(""" SELECT * FROM api_values_tables """).show()

+----------+----------+
|      date|product_id|
+----------+----------+
|2021-07-06|     42662|
|2021-07-06|     45474|
+----------+----------+



In [27]:
spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW api_values_tables2
    USING json
    OPTIONS (path '/test/api_values1.json')
""")

DataFrame[]

In [28]:
spark.sql(""" SELECT * FROM api_values_tables2 """).show()

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).json(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).json(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).json(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().